<a href="https://colab.research.google.com/github/mukulsn/Machine-Learning/blob/main/Tensorflow%20Developer%20Certificate/NLP_Text_classification_using_Hugginface_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# yesterday completed daniel bourke course NLP script
> now i will try to incorporate hugging face models and check the accuracy and F1 score of the models

In [ ]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-12-03 16:19:01--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-03 16:19:01 (108 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-12-03 16:19:13--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.97.207, 74.125.196.207, 74.125.139.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.97.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2023-12-03 16:19:13 (44.2 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df = train_df.rename(columns={'target':'label'})
test_df = test_df.rename(columns={'target':'label'})
train_df

,id,keyword,location,text,label
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,label
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# How many examples of each class?
train_df.label.value_counts()


0    4342
1    3271
Name: label, dtype: int64

In [ ]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [ ]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text','label']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}', "(real disaster)" if target > 0 else "(not real disaster)")
  print(f'Text:\n{text}\n')
  print("----\n")

Target: 0 (not real disaster)
Text:
Whirlwind weekend #1 starts in 12 hours #cantwaittoplayinminneapolis

----

Target: 0 (not real disaster)
Text:
@SunderCR two hours of Sandstorm remixes. All merged together. No between-song silence.

----

Target: 1 (real disaster)
Text:
Governor Allows Parole for School Bus Hijacker http://t.co/DzlPNP399x

----

Target: 1 (real disaster)
Text:
So a train derailed and instead of me getting to work early like I would've I'm now late as fuck

----

Target: 1 (real disaster)
Text:
Hinton city officials confirm multiple buildings on fire currently evacuating people on Main Street. We're on our way @OKCFOX

----



In [ ]:
# splitting dataset
from sklearn.model_selection import train_test_split

# Use tran_test_split to split training datainto training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), # to convert into numpy arrays for tensorflow
                                                                            train_df_shuffled['label'].to_numpy(),
                                                                            test_size=0.1, # 10% of validation set
                                                                            random_state=42)

In [ ]:
# check the lengths
len(train_sentences), len(train_labels),len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

# huggingface

In [ ]:

!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
train_df = pd.DataFrame({'text':train_sentences,'label':train_labels})
val_df = pd.DataFrame({'text':val_sentences,'label':val_labels})

In [ ]:
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(train_df)
val = Dataset.from_pandas(val_df)
val

Dataset({
    features: ['text', 'label'],
    num_rows: 762
})

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples['text'], truncation=True)

In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_val = val.map(preprocess_function, batched=True)


Map:   0%|          | 0/6851 [00:00<?, ? examples/s]

Map:   0%|          | 0/762 [00:00<?, ? examples/s]

In [ ]:
tokenized_train[10]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

In [ ]:
import evaluate
accuracy=evaluate.load('accuracy')

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0:'NEGATIVE',1:'POSITIVE'}
label2id = {'NEGATIVE':0,'POSITIVE':1}

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_train)//batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',num_labels=2,id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
!pip install datasets

In [ ]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 6851
})

In [ ]:
# convert to tf tensors
tf_train_set = model.prepare_tf_dataset(
    tokenized_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_val,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
# configure for training
import tensorflow as tf
model.compile(optimizer=optimizer) # No loss argument becase by default transformer model have loss function

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
# where to push model
from transformers.keras_callbacks import PushToHubCallback
push_to_hub = PushToHubCallback(
    output_dir='tweet_disaster_model',
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/mukulsn/tweet_disaster_model into local empty directory.


In [ ]:
callbacks=[metric_callback,push_to_hub]


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3
428/428 [==============================] - 102s 170ms/step - loss: 0.4381 - val_loss: 0.3847 - accuracy: 0.8333
Epoch 2/3
428/428 [==============================] - 76s 178ms/step - loss: 0.3213 - val_loss: 0.3963 - accuracy: 0.8346
Epoch 3/3
428/428 [==============================] - 81s 189ms/step - loss: 0.2409 - val_loss: 0.4119 - accuracy: 0.8307


In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

# Loading model custom model from huggingface

In [ ]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model='mukulsn/tweet_disaster_model')
classifier(text)

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at mukulsn/tweet_disaster_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9939502477645874}]

In [ ]:
text="No I don't like cold!"
classifier(text)

[{'label': 'NEGATIVE', 'score': 0.951302707195282}]

In [ ]:
test_df.loc[:,'text'].to_numpy()

array(['Just happened a terrible car crash',
       'Heard about #earthquake is different cities, stay safe everyone.',
       'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all',
       ..., 'Green Line derailment in Chicago http://t.co/UtbXLcBIuY',
       'MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3',
       '#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm'],
      dtype=object)

In [ ]:
!pip install tqdm

In [ ]:
# calculate the time of predictions
import time
# from tqdm import tqdm
from tqdm.auto import tqdm
# predictions = []
a = None
def pred_timer(samples):
  start_time = time.perf_counter()
  predictions = [classifier(x) for x in tqdm(samples)]
  end_time = time.perf_counter()
  total_time = end_time-start_time
  time_per_pred = total_time/len(samples)
  a = predictions
  return (total_time, time_per_pred,predictions)

In [ ]:
test_df.shape, val_df.shape

((3263, 4), (762, 2))

In [ ]:

predictions = [classifier(x) for x in tqdm(test_df.loc[:,'text'].to_numpy())]

In [ ]:
%time
pred_timer(test_df.loc[:,'text'].to_numpy())

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


100%|██████████| 3263/3263 [14:29<00:00,  3.75it/s]


(869.9750269219999, 0.26661815106405146)

In [ ]:
a,b,c = pred_timer(val_df.loc[:,'text'].to_numpy())

  0%|          | 0/762 [00:00<?, ?it/s]

  1%|▏         | 10/762 [05:37<7:03:26, 33.79s/it]


In [ ]:
# creating evaluation function
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score
  """
  # calculate model accuracy
  model_accuracy = accuracy_score(y_true,y_pred) * 100
  # calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true,y_pred, average='weighted')
  model_results={'accuracy':model_accuracy,
                 'precision':model_precision,
                 'recall':model_recall,
                 'f1':model_f1}
  return model_results

In [ ]:
calculate_results(y_true= val_df['label'].to_numpy(),
                  y_pred= [(round(x[0]['score'])) for x in c])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 45.66929133858268,
 'precision': 0.2085684171368343,
 'recall': 0.4566929133858268,
 'f1': 0.2863587997446265}

In [ ]:
_a = val_df['label'].to_numpy()

In [ ]:
c[0][0]['score']

0.789932370185852

In [ ]:
_ = [(round(x[0]['score'])) for x in c]

In [ ]:
def print_score(ind):
  return (_a[ind],_[ind])

In [ ]:
print_score(2)

(1, 1)

In [ ]:
len(_),len(_a)

(762, 762)

In [ ]:
temp = [1 if a==b else 0 for a,b in zip(_a,_)]
temp

In [ ]:
sum(temp)

348

In [ ]:
348/762

0.4566929133858268

# trying manual method

In [ ]:
text = [x for x in val_df['text']]